<a href="https://colab.research.google.com/github/NARizzardi/machine-learning/blob/main/Trabalho/Trabalho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import kagglehub
import cv2
import os

path = kagglehub.dataset_download("zlatan599/mushroom1")

mushroomdf_val = pd.read_csv(f"{path}/val.csv") # nao irei usar provavelmente para a analise mas ja deixo ai
mushroomdf_test = pd.read_csv(f"{path}/test.csv") # val kinda related to test
mushroomdf_train = pd.read_csv(f"{path}/train.csv").drop_duplicates(subset=["image_path"])

mushroomdf_val['image_path'] = mushroomdf_val['image_path'].str.replace('/kaggle/working','/kaggle/input/mushroom1')
mushroomdf_train['image_path'] = mushroomdf_train['image_path'].str.replace('/kaggle/working','/kaggle/input/mushroom1')
mushroomdf_test['image_path'] = mushroomdf_test['image_path'].str.replace('/kaggle/working','/kaggle/input/mushroom1')

paths_val = mushroomdf_val.image_path
paths_train = mushroomdf_train.image_path
paths_test = mushroomdf_test.image_path

paths = list(paths_train) + list(paths_test)


# Mais velocidade usando multiprocessamento
from joblib import Parallel, delayed

X_train = []
y_train = []
X_test = []
y_test = []

# These can be found in /content
if os.path.exists("X_train.npy") and os.path.exists("y_train.npy") and os.path.exists("X_test.npy") and os.path.exists("y_test.npy"):
    X_train = np.load("X_train.npy")
    y_train = np.load("y_train.npy", allow_pickle=True)
    X_test = np.load("X_test.npy")
    y_test = np.load("y_test.npy", allow_pickle=True)
    print("Loaded from files")
else:
  print("Loading images...")

  def loadImg(path):
    try:
      img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
      img = cv2.resize(img, (64, 64))
      return img.flatten() / 255.0
    except Exception as e:
      print(f"{e} - Error loading {path}")
      return None

  results = Parallel(n_jobs=-1, prefer="threads")(
      delayed(loadImg)(p)
      for p in paths
  )

  #Removendo cargas falhas
  results = [img for img in results if img is not None]

  X_train = np.array(results[:len(paths_train)])
  y_train = mushroomdf_train.label.to_numpy()

  X_test = np.array(results[len(paths_train):])
  y_test = mushroomdf_test.label.to_numpy()

  # save
  np.save("X_train.npy", X_train)
  np.save("y_train.npy", y_train)
  np.save("X_test.npy", X_test)
  np.save("y_test.npy", y_test)

# ~4 minutos se kaggle nao presente
# ~8-10mins pra tudo ser alocado
# ~18segundos-1minuto  se tudo presente

# Algumas verificacoes para ver se ta tudo certo, pode ser comentado
print(f"Total train paths: {len(paths_train)}, total test paths: {len(paths_test)}")
print(f"Images on train loaded: {len(X_train)}, labels loaded: {len(y_train)}, images on test: {len(X_test)}, labels on test: {len(y_test)}")

print(f"Shape of X_train: {X_train.shape} and of y_train: {y_train.shape}")
print(f"Shape of X_test: {X_test.shape} and of y_test: {y_test.shape}")

100%|██████████| 11.3G/11.3G [01:46<00:00, 114MB/s]

Extracting files...


Streaming output truncated to the last 5000 lines.
OpenCV(4.12.0) /io/opencv/modules/imgproc/src/resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
 - Error loading /kaggle/input/mushroom1/merged_dataset/Hypogymnia physodes/26802medium.jpeg
OpenCV(4.12.0) /io/opencv/modules/imgproc/src/resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
 - Error loading /kaggle/input/mushroom1/merged_dataset/Suillus spraguei/Suillus spraguei_259.jpg
OpenCV(4.12.0) /io/opencv/modules/imgproc/src/resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
 - Error loading /kaggle/input/mushroom1/merged_dataset/Psilocybe cyanescens/Psilocybe cyanescens_38.jpg
OpenCV(4.12.0) /io/opencv/modules/imgproc/src/resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
 - Error loading /kaggle/input/mushroom1/merged_dataset/Vulpicida pinastri/32674medium.jpeg
OpenCV(4.12.0) /io/opencv/modules/imgproc

In [2]:
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform it
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data using the scaler fitted on the training data
X_test_scaled = scaler.transform(X_test)

print("Data scaled successfully. Here are the shapes of the scaled data:")
print(f"Shape of X_train_scaled: {X_train_scaled.shape}")
print(f"Shape of X_test_scaled: {X_test_scaled.shape}")


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split


def svm(nucleo, subset_size):
  svm_model = SVC(kernel=nucleo, random_state=42, verbose=True)

  X_train_subset, _, y_train_subset, _ = train_test_split(
      X_train, y_train, train_size=subset_size, stratify=y_train, random_state=42
  )

  print(f"Treinando modelo SVM com um subconjunto de {len(X_train_subset)} amostrar...")
  svm_model.fit(X_train_subset, y_train_subset)
  print("Treinamento do modelo SVM completo.")
  return svm_model

svm_model = svm('rbf', 0.25)

In [ ]:

print("Fazendo predições no conjunto de teste...")
y_pred = svm_model.predict(X_test)
print("Predições completas.")

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAcurária: {accuracy:.4f}")

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))


In [ ]:
from sklearn.manifold import TSNE

train_reducer = TSNE(random_state=2025, verbose=True, n_jobs=1, perplexity=20.0, init='pca')
train_df[['x', 'y']] = train_reducer.fit_transform(X=X_train['value'].apply(func=pd.Series))